# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [3]:
%pip install duckdb leafmap
%pip install folium matplotlib mapclassify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.6 MB/s eta 0:00:00


Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [4]:
import leafmap
import duckdb

url = "https://open.gishub.org/data/duckdb/nyc_data.zip"
leafmap.download_file(url, unzip=True, overwrite=True)


con = duckdb.connect("nyc_data.db")
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
con.install_extension("spatial")
con.load_extension("spatial")

con.sql("SHOW TABLES;")


Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.zip
To: /content/nyc_data.zip
100%|██████████| 8.73M/8.73M [00:00<00:00, 173MB/s]


Extracting files...


┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

In [20]:
import os

extract_path = "/content/nyc_data"
t
# 检查路径是否存在
if os.path.exists(extract_path):
    if os.path.isfile(extract_path):  # 该路径是一个文件
        print("⚠️ `/content/nyc_data` 已经是一个文件，先删除")
        os.remove(extract_path)  # 删除该文件
    elif os.path.isdir(extract_path):  # 该路径是一个目录
        print("✅ `/content/nyc_data` 已经是一个文件夹")
else:
    print("🔍 `/content/nyc_data` 目录不存在，可以安全解压")

✅ `/content/nyc_data` 已经是一个文件夹


In [21]:
import zipfile

zip_path = "/content/nyc_data.zip"
extract_path = "/content/nyc_data"

# 重新解压 ZIP 文件
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# 查看解压后的内容
print("📂 解压后文件列表:", os.listdir(extract_path))

📂 解压后文件列表: ['nyc_census_blocks.dbf', 'nyc_homicides.dbf', 'nyc_neighborhoods.prj', 'nyc_neighborhoods.shp', 'nyc_streets.dbf', 'nyc_homicides.shx', 'nyc_streets.shx', 'nyc_subway_stations.prj', 'nyc_census_sociodata.sql', 'nyc_homicides.shp', 'nyc_subway_stations.dbf', 'nyc_streets.shp', 'nyc_census_blocks.shp', 'nyc_census_blocks.shx', 'README.txt', 'nyc_streets.prj', 'nyc_neighborhoods.dbf', 'nyc_subway_stations.shx', 'nyc_neighborhoods.shx', 'nyc_census_blocks.prj', 'nyc_homicides.prj', 'nyc_subway_stations.shp']


**Create tables which could be used afterwards.**

In [35]:
shp_file = "/content/nyc_data/nyc_neighborhoods.shp"
shp_file1 = "/content/nyc_data/nyc_subway_stations.shp"
shp_file2 = "/content/nyc_data/nyc_streets.shp"
shp_file3 = "/content/nyc_data/nyc_census_blocks.shp"

con.execute(f"""
    CREATE TABLE nyc_census_blocks AS
    SELECT * FROM ST_Read('{shp_file3}');
""")

print(con.execute("SHOW TABLES;").fetchdf())
print(con.execute("SELECT * FROM nyc_census_blocks LIMIT 5").fetchdf())

                  name
0    nyc_census_blocks
1    nyc_neighborhoods
2          nyc_streets
3  nyc_subway_stations
             BLKID  POPN_TOTAL  POPN_WHITE  POPN_BLACK  POPN_NATIV  \
0  360850009001000          97          51          32           1   
1  360850020011000          66          52           2           0   
2  360850040001000          62          14          18           2   
3  360850074001000         137          92          12           0   
4  360850096011000         289         230           0           0   

   POPN_ASIAN  POPN_OTHER       BORONAME  \
0           5           8  Staten Island   
1           7           5  Staten Island   
2          25           3  Staten Island   
3          13          20  Staten Island   
4          32          27  Staten Island   

                                                geom  
0  [2, 4, 0, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151, 8...  
1  [2, 4, 0, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72, ...  
2  [2, 4, 0, 0, 0, 0, 0, 0, 82, 22

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [25]:
con.sql("SELECT s.name AS subway_station FROM nyc_neighborhoods AS n JOIN nyc_subway_stations AS s ON ST_Contains(n.geom, s.geom) WHERE n.name = 'Little Italy';")

┌────────────────┐
│ subway_station │
│    varchar     │
├────────────────┤
│ Spring St      │
└────────────────┘

In [27]:
italy_station_df = con.sql("SELECT s.name AS subway_station, s.routes as subway_route, st_astext(s.geom) as geometry FROM nyc_neighborhoods AS n JOIN nyc_subway_stations AS s ON ST_Contains(n.geom, s.geom) WHERE n.name = 'Little Italy';").df()
italy_station_df.head()

,subway_station,subway_route,geometry
0,Spring St,6,POINT (584696.3146033001 4508413.075791954)


In [28]:
italy_station_df = leafmap.df_to_gdf(italy_station_df, src_crs="EPSG:26918", dst_crs="EPSG:4326")
italy_station_df.head()

,subway_station,subway_route,geometry
0,Spring St,6,POINT (-73.99713 40.72229)


In [29]:
italy_station_df.explore()

Spring St is in " Little Italy" and subway route 6 is in Little Italy.

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [31]:
italy_station_df = con.sql("SELECT distinct on (n.name) n.name AS neighborhoods, s.routes as subway_route, st_astext(s.geom) as geometry FROM nyc_neighborhoods AS n JOIN nyc_subway_stations AS s ON ST_Contains(n.geom, s.geom) WHERE s.routes = '6';").df()
italy_station_df.head()

,neighborhoods,subway_route,geometry
0,Chinatown,6,POINT (584414.3752830802 4507981.764333538)
1,Greenwich Village,6,POINT (584901.3978172147 4508818.983088133)
2,Midtown,6,POINT (586780.4926467581 4512304.301414792)
3,Upper East Side,6,POINT (587430.060968678 4513505.55859977)
4,Yorkville,6,POINT (588783.3683566097 4516014.70932135)


The neighborhoods served by the 6-train are Chinatown, Greenwich Village, Midtown and Upper East Side as well as Yorkville.

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

nyc_census_blocks
nyc_neighborhoods

In [41]:
print(con.execute("SELECT * FROM nyc_census_blocks LIMIT 5").fetchdf())
print(con.execute("SELECT * FROM nyc_neighborhoods where Name = 'Battery Park' LIMIT 5").fetchdf())

             BLKID  POPN_TOTAL  POPN_WHITE  POPN_BLACK  POPN_NATIV  \
0  360850009001000          97          51          32           1   
1  360850020011000          66          52           2           0   
2  360850040001000          62          14          18           2   
3  360850074001000         137          92          12           0   
4  360850096011000         289         230           0           0   

   POPN_ASIAN  POPN_OTHER       BORONAME  \
0           5           8  Staten Island   
1           7           5  Staten Island   
2          25           3  Staten Island   
3          13          20  Staten Island   
4          32          27  Staten Island   

                                                geom  
0  [2, 4, 0, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151, 8...  
1  [2, 4, 0, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72, ...  
2  [2, 4, 0, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55, ...  
3  [2, 4, 0, 0, 0, 0, 0, 0, 204, 85, 13, 73, 103,...  
4  [2, 4, 0, 0, 0, 0, 0, 0, 107, 247, 1

In [48]:
Park_station_df = con.sql("SELECT sum(a.popn_total) AS population, b.NAME as neighborhood FROM nyc_census_blocks a JOIN nyc_neighborhoods b ON ST_Intersects(a.geom, b.geom) WHERE b.name = 'Battery Park' Group by b.name ;").df()
Park_station_df.head()

,population,neighborhood
0,17153.0,Battery Park


17153 people would have to be evacuated.

4. **What neighborhood has the highest population density (persons/km2)?**


In [54]:
neighborhood_df = con.sql("SELECT round(sum(a.popn_total)/(ST_Area(b.geom)/1000000),3) as population_density, sum(a.popn_total) AS population, b.NAME as neighborhood, ST_Area(b.geom)/1000000 as km2 FROM nyc_census_blocks a JOIN nyc_neighborhoods b ON ST_Intersects(a.geom, b.geom) Group by b.name,b.geom order by population_density desc;").df()
neighborhood_df.head()

,population_density,population,neighborhood,km2
0,68435.133,22460.0,North Sutton Area,0.328194
1,50404.483,82266.0,East Village,1.632117
2,48825.181,16209.0,Chinatown,0.331980
3,48543.725,18763.0,Carnegie Hill,0.386518
4,48524.488,203741.0,Upper East Side,4.198725


North Sutton Area has the highest population density.

When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [5]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00


In [6]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [7]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

In [3]:
import leafmap
import duckdb
import pandas as pd
import matplotlib
import mapclassify

url = "https://storage.googleapis.com/qm2/casa0025_ships.csv"
# df = pd.read_csv(url, sep=",", header=0)  # Corrected header argument
# print(df.head())  # Display first few rows
# print(df.info())

In [4]:
import pandas as pd
import duckdb
import geopandas as gpd
from shapely.wkt import loads
from shapely.geometry import Point

# Load the CSV file
df = pd.read_csv(url)

# Check if 'geom' column is in WKT format
if "geom" in df.columns and df["geom"].dtype == "object":
    df["geom"] = df["geom"].apply(lambda x: loads(x))  # Convert WKT to geometry
else:
    # If geom is missing, create it from lat/lon
    df["geom"] = df.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry="geom", crs="EPSG:4326")


In [5]:
# Connect to DuckDB and enable spatial extension
con = duckdb.connect(":memory:")  # Use an in-memory database
con.execute("INSTALL spatial;")  # Install spatial extension
con.execute("LOAD spatial;")  # Load spatial extension

In [6]:
# Convert geometry to WKT for storage
gdf["geom"] = gdf["geom"].apply(lambda x: x.wkt)

# Register Pandas DataFrame in DuckDB
con.register("ais_table", gdf)

In [7]:
con.execute("""
    CREATE TABLE ais AS
    SELECT
        date,
        vesselid,
        sog,
        lat,
        lon,
        ST_GeomFromText(geom, 4326) AS geom
    FROM ais_table;
""")

BinderException: Binder Error: No function matches the given name and argument types 'ST_GeomFromText(VARCHAR, INTEGER_LITERAL)'. You might need to add explicit type casts.
	Candidate functions:
	ST_GeomFromText(VARCHAR) -> GEOMETRY
	ST_GeomFromText(VARCHAR, BOOLEAN) -> GEOMETRY

LINE 9:         ST_GeomFromText(geom, 4326) AS geom
                ^

In [23]:
con.sql("DROP TABLE IF EXISTS ais;")
con.sql("DROP TABLE IF EXISTS vinfo;")

con.execute(f"""
    CREATE TABLE ais AS
    SELECT date,
           vesselid,
           sog,
           lat,
           lon,
           ST_GeomFromText(CONCAT('POINT(', lon, ' ', lat, ')')) as geom
    FROM df;
""")

con.execute(f"""
    CREATE TABLE vinfo AS
    SELECT DISTINCT
    vesselid,
    vessel_name,
    vsl_descr,
    dwt,
    v_length,
    draught FROM df;
""")


print(con.execute("SHOW TABLES;").fetchdf())
print(con.execute("SELECT * FROM ais LIMIT 5").fetchdf())


    name
0    ais
1  vinfo
                  date  vesselid  sog        lat        lon  \
0  2022-07-25 02:53:29    350053  5.2  45.151777  36.513327   
1  2022-07-25 03:09:37    350053  0.7  45.146487  36.520780   
2  2022-07-25 03:13:58    350053  0.7  45.146218  36.521965   
3  2022-07-25 04:16:06    350053  0.1  45.145058  36.522020   
4  2022-07-25 05:20:17    350053  0.0  45.144933  36.521848   

                                                geom  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  


In [28]:
con.sql("DROP TABLE IF EXISTS ais;")
con.sql("DROP TABLE IF EXISTS vinfo;")

con.execute("""
    CREATE TABLE ais (
        date STRING,
        vesselid INTEGER,
        sog DOUBLE,
        geom GEOMETRY
    );
""")  # Define geom as GEOMETRY type

con.execute(f"""
    INSERT INTO ais
    SELECT date,
           vesselid,
           sog,
           geom
    FROM df;
""")  # Insert data with ST_GeomFromText


con.execute(f"""
    CREATE TABLE vinfo AS
    SELECT DISTINCT
    vesselid,
    vessel_name,
    vsl_descr,
    dwt,
    v_length,
    draught FROM df;
""")


print(con.execute("SHOW TABLES;").fetchdf())
print(con.execute("SELECT * FROM ais LIMIT 5").fetchdf())

    name
0    ais
1  vinfo
                  date  vesselid  sog  \
0  2022-07-25 02:53:29    350053  5.2   
1  2022-07-25 03:09:37    350053  0.7   
2  2022-07-25 03:13:58    350053  0.7   
3  2022-07-25 04:16:06    350053  0.1   
4  2022-07-25 05:20:17    350053  0.0   

                                                geom  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  


## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

In [ ]:
df = con.sql("""
    SELECT a1.vesselid AS ship1,
           a2.vesselid AS ship2,
           a1.date AS start,
           a2.date AS end
    FROM ais AS a1
    JOIN ais AS a2
    ON ST_DWithin(a1.geom, a2.geom, 500)
    AND ABS(EXTRACT(EPOCH FROM (CAST(a2.date AS TIMESTAMP) - CAST(a1.date AS TIMESTAMP)))) > 7200
    AND a1.vesselid <> a2.vesselid;""").df()
df.head()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))